## 9.2 Motivation

머신러닝 시스템을 향상시키기 위해 convolution은 3가지의 주요한 아이디어를 사용합니다. sparse interactions, parameter sharing, equivariant representaton , 또한 convolution은 variable size input에 대해 시스템이 작동 할 수 있게 합니다.

<img src='https://image.slidesharecdn.com/understandingconvolutionalneuralnetworks-161220064100/95/understanding-convolutional-neural-networks-19-638.jpg?cb=1482216923'>

<b>sparse interaction</b> : 인풋보다 작은 커널을 사용함으로써 생기는 현상으로 적은 파라미터를 사용하게 하며 모델의 메모리 사용량을 줄여주고 통계적 효율성을 높여줍니다. 또한 연산량도 줄여줍니다.

<b> parameter sharing</b> : 파라미터 수를 줄여주어 파라미터를 저장해야할 메모리를 줄여주고, feed forward의 연산량은 바뀌지 않는다. 파라미터가 줄었음으로 통계적 효율성이 높아진다.

<b>equivariance to traslation</b> : parameter sharing의 형태는 CNN이 equivariance to translation 하게 만듭니다. 함수가 equivariant하다는 의미는 인풋 이 바뀌면 아웃풋도 인풋이 바뀐만큼 바뀌어야 한다는 것입니다. 함수 $f(x)$ 가 g에 대해 equivariant 하게 되면 $f(g(x)) = g(f(x))$ 가 됩니다. convolution의 경우에 g가 어떠한 translation function이라고 생각해보면 convolution function은 g라는 translation function에 대해 equirvariant하게 됩니다. 예를 들어 I라는 함수가 이미지의 brightness(픽셀값) 을 나타내는 함수라고 가정하고 g를 I를 I\`으로 매핑하는 함수라고 생각합시다. 이때 I\`(x,y) = I(x-1,y) 즉 모든 픽셀을 한칸씩 오른쪽으로 shift하는 함수입니다. I\`, I 에 convolution을 적용함 값을 모두 같게 됩니다. 시계열 데이터를 처리할때 이러한 equivariance to translation의 의미는 인풋에 다른 feature가 나타날 때를 보여주느 일종의 time-line을 생성하게 됩니다. (?? 무슨의미지) 시간축에서의 하나의 이벤트를 옮기게 되면 같은 representation이 시간축 만큼만 이동되어 output에 나타나게 됩니다. 이미지에서도 비슷하게 convolution은 input image에서의 feature를 나타내는 2d map을 생성하게 됩니다. 인풋에서의 object가 움직이면 아웃풋에서의 representation도 객체가 움직인 만큼 움직여서 나타나게 됩니다. 이는 어떤 인접한 pixel을 인풋으로 받는 일종의 local function이 인풋의 각기 다른 location에 적용될때 유용하게 됩니다. 예를들어 이미지 처리에서 cnn의 첫번째 레이어는 edge를 찾는데 유용합니다. 같은 패턴의 edge는 이미지 전체에서도 나타날 수 잇고 이는 location간 파라미터를 공유하는것이 유용하다는 것을 보여줍니다. 어떤 경우에는 전체 이미지에서 파라미터를 공유하고 싶지 않을 수 있습니다. 예를 들어 개인의 face가 crop, centered된 이미지를 처리한다고 가정해봅시다. 이러한 경우 각기 다른 location에서 다른 feature를 추출하길 원할 수 있습니다. 이미지의 top을 처리하는 네트워크는 눈썹과 같은 feature를 찾아야 하고 bottom쪽을 처리하는 feature는 턱과 같은 다른 feature를 추출해야 합니다.

## 9.3 Pooling

pooling은 인풋의 small translation에 대해 근사적으로 invariant한 representation을 가지도록 해줍니다. invariance to translation의 이미는 우리가 인풋을 약간 translate하면 대부분의 아웃풋들은 바뀌지 않아야 합니다. <b>invariance to local translation은 feature가 어디에 있는지 보다 어떤 feature가 있는지에 관심 있을때 더 유용합니다</b>

예를 들어 이미지안에 얼굴이 있는지를 결정할때 우리는 눈의 위치가 어디인지에 대해 아주 정확히 알 필요는 없습니다. 단지 우리는 얼굴의 왼쪽과 오른쪽 쯔음에 눈이 있다는 것만 알면 됩니다. 다른 경우에는 feature의 위치를 보존하는 것이 매우 중요할 수가 있습니다. 예를 들어 우리가 특정한 방향의 두 edge가 만나는 corner를 찾고 싶어한다고 합시다. 이런 경우 우리는 edge featuree의 위치를 정확히 알아야 어디에서 만나는지를 알아낼 수 있습니다. 

<b> pooling을 사용하는 것은 레이어가 학습하는 함수가 invariant to small translations 이라는 infinitely strong prior를 추가하는 것으로 볼 수 있습니다.</b> 이러한 가정이 올바르다면 이는 네트워크의 통계적 효율성을 향상시켜줄 것입니다.

spatial region에 대한 pooling은 invariance to translation을 만들어 줍니다. 만약 우리가 파라미터를 공유하지 않는 convolution의 output에 풀링을 하게 되면 feature는 어떤 transformation에 대해 invariant해야 할지를 배우게 됩니다. <b>즉 multi-channel pooling은 invariant to other transformation을 학습할때만 사용된다.</b>

많은 task에서 풀링은 variable size input을 다루는데 종종 사용됩니다. 예를 들어 우리가 variable size image를 claissfy하고 싶다고 해봅시다. classfication layer의 인풋은 고정된 크기여야 합니다. 이는 pooling region offset을 조절함으로써 해결할 수 있고 classfication layer는 항상 고정된 크기를 인풋으로 받게 됩니다. 예를 들어 네트워크의 마지막 pooling layer가 4개의 요약된 통계치를 출력하도록 정의될 수 있습니다. 각각의 하나의 output은 이미지의 4분면에 해당하게 되며 이는 이미지 크기와 상관이 없게 됩니다.

### Pooling
 - 인접한 픽셀에 대한 summary statics 를 제공.
 - invariant to small translation 을 제공
 - 다운사이즈를 통해 연산량의 이점
 - cross channel pooling을 하면 어떤 transformation에 invariant할지를 학습 할 수 있음있

## 9.4 Convolution and Pooling as an Infinitely Strong Prior

prior는 prior 확률분포를 어떻게 구성했는지에 따라 weak, strong prior로 생각 될 수 있습니다. 높은 분산을 갖는 가우시안 분포와 같은 high entropy prior distribution은 weak prior 입니다. 이러한 prior는 데이터가 파라미터를 더 자유롭게 혹은 덜 자유롭게 움직이도록 합니다. 낮은 분산을 갖는 가우시안 분포오 같은 low entropy prior distribution은 파라미터가 어떻게 업데이트되어야 할지에 대해 더 적극적인 역할을 하게 됩니다.

infinitely strong prior는 몇몇 파라미터를 zero probability로 만들며 이러한 파라미터 값이 불가능 하다고 말합니다. 이는 데이터와 파라미터간의 관계는 상관하지 않습니다.

우리는 cnn을 fc-net과 동일하지만 웨이트에 대해 infinitely strong prior를 가진 형태로 볼 수 있습니다. 이러한 infinitely strong prior는 하나의 히든 유닛에 대한 웨이트가 인접한 유닛과 같아야 하며 space상에서 shift 되어야 한다는 prior를 가진 형태입니다. 이러한 prior는 그 히든 유닛에 할당된 작은 receptive field를 제외한 다른 웨이트들은 0이여야 한다고 말합니다. 다시 말하자면 cnn은 레이어의 파라미터에 대한 infinitely strong prior 를 도입한 것으로 생각 할 수 있습니다. 이 priro는 레이어가 학습해야하는 함수가 오직 local interaction 만을 포함하고 equivariant to translation 해야 하도록 강제합니다. 비슷하게 pooling은 각 히든 유닛이 invariant to small translation 해야 한다는 infiniely strong prior입니다.

물론 cnn을 prior가 있는 fc-layer 형태로 구현하는 것은 연산을 매우 낭비하는 것입니다. 하지만 이러한 cnn을 prior가 있는 fc-layer 형태로 보는 것은 cnn이 어떻게 동작하는지에 대한 인사이트를 줍니다.

첫번째 인사이트는 convolution, pooling이 underfitting의 원인이될 수 있습니다. 다른 prior와 마찬가지로 convolution, pooling은 이들의 prior가 정확할 때만 잘 작동합니다. 만약 우리의 task가 정확한 spatial information을 보존해야 한다면 pooling을 사용하는 것은 트레이닝 에러를 높일 수 있습니다. 몇몇 cnn 구조들은 cnn 채널의 부분에만 pooling을 적용하여 highly invarinat feature와 prior가 정확하지 않을떄의 feature를 둘다 뽑아냅니다. task가 local한 정보가 아닌 더 멀리 잇는 픽셀에 대한 정보를 필요로 한다면 convolution의 prior는 적절하지 않을 것입니다.

### prior of convolution, pooling
- convolution : equvariant to translation, local interactions
- pooling : invariant to small translation prior



## 9.5 Variants of the Basic Convolution Function

뉴럴네트워크에서의 convolution을 설명할때 수학적으로 보통 사용되는 표준적인 discreate convolution operation에 대해 언급하지 않았습니다. 실제로 뉴럴네트워크에서 사용되는 convolution은 약간 다릅니다. 이 섹션이서 차이점을 설명하고 뉴럴네트워크에서 사용되는 convolution의 특징에 대해 설명합니다.

먼저 뉴럴네트워크에서의 convolution은 병렬적으로 수행되는 여러개의 convolution의 의미를 가집니다. 하나의 커널을 가지는 convolution은 여러 location의 한 종류의 feature만 추출 할 수 있습니다. 뉴럴네트워크에서 우리가 원하는 것은 하나의 레이어가 여러개의 feature를 뽑는 것을 원합니다. 

또한 cnn에서의 input은 종종 grid of real value가 아니며 grid of vector-value 를 가지게 됩니다. 예를 들면 칼라 이미지는 각 픽셀마다 rgb에 해당하는 3개 채널을 가지고 있습니다. multi-layer cnn의 경우 두번째 레이어의 인풋은 첫번째 레이어의 아웃풋이 되며 두번째 레이어는 각 location마다 여러개의 convolution을 가진 형태가 됩니다. cnn으로 이미지를 처리 할대 보통 인풋과 아웃풋을 3D tensor로 생각하며 첫번째 차원은 다른 채널들을 나타내며 나머지 2차언은 height, width를 나타냅니다. 소프트웨어 구현체에서는 보통 batch를 사용하여 이들은 3D가 아닌 4D Tensor가 되며 4번째 축은 batch size를 나타냅니다. 여기서는 설명을 위해 batch 축을 제외하고 설명합니다.

cnn은 보통 multi-channel convolution으로 사용되기 때문에 이러한 convolution은 더이상 kernel-flip을 사용하더라도 commutative 가 성립하지 않습니다. multi-channel convolutiona은 아웃풋, 인풋의 차원이 같을때만 교환법칙이 성립합니다.

우리가 4D kernel tensor $\mathbf{K}$ 를 가지고 있다고 해봅시다. K_i,j,k,l 은 아웃풋의 i 채널, 인풋의 j 채널, k 행, l 열에 대해 연결되어 집니다. 인풋이 $\mathbf{V}$ 이며 V_i,j,k 가 i 채널, j 행, k 열의 값을 나타낸다고 합시다. 이러한 커널과 인풋에 대한 convolution output $\mathbf{Z}$ 는 $\mathbf{V}$ 와 같은 크기를 가지게 됩니다. flip-kernel 이 없다고 생각했을때 Z는 다음과 같이 계산됩니다.

## $\mathbf{Z}_{i,j,k} = \sum_{i,m,n}\mathbf{V}_{l,j+m-1, k+n-1} \mathbf{K}_{i,l,m,n}$

l,m,n에 대해 summation을 취할때 summation이 유효한 모든 값들에 대해서 summation이 됩니다. (즉 커널이... 0이 아닌 값에 대해서만... summation이 된다는 뜻인듯) 선형대수의 notation을 사용하면 array의 첫번째 요소는 1이라는 index를 가지며 이 때문에 위의 식에 -1 의 텀이 들어가야 합니다. C, python 과 같은 언어에서는 0부터 index가 시작되 -1이 없어도 됩니다. 

연산량을 줄이기 위해 커널에서의 몇몇 position에서는 summation을 하지 않고 싶을 수 있습니다. 이러한 것을 convolution function에 의해 down sampling 된 것으로 생각 할 수 있습니다. 만약 우리가 매번 s pixel만을 샘플링 하고 싶다면 downsampled convolution을 다음과 같이 정의할 수 있습니다. 

## $\mathbf{Z}_{i,j,k} = \sum_{i,m,n}\mathbf{V}_{l,(j-1)*s, (k-1)*s+n} \mathbf{K}_{i,l,m,n}$

우리는 s를 stride라고 부릅니다. 각 방향으로 다른 stride를 적용하는 것도 가능합니다. <b> stride 1 이상의 downsampled convolution은 수학적으로  1 stride convolution 후에 downsampling 을 적용하는 것과 같습니다</b>

cnn의 구현중 중요한 것중 하나는 인풋에 zero-pad를 하는 것입니다. 이러한 zero-pad가 없이는 각 cnn 레이어마다 kernel width-1, height-1 만큼 사이즈가 줄어들 것입니다. 인풋에 zero pad를 함으로서 커널의 크기와 아웃풋의 크기가 독립적으로 만들 수 있습니다. zero padding이 없다면 spatial extent가 빠르게 줄어들 것이며 작은 커널을 사용해야만 합니다. 이러한 경우 네트워크의 representation power를 매우 제한시키게 됩니다.

 - valid convolution : zero padding 이 없는 형태의 covolution
 - same convolution : 인풋과 아웃풋의 크기가 같도록 zero-padding을 추가, 가장자리 쪽의 인풋 픽셀이 아웃풋 픽셀에 끼치는 영향이 적어진다. 
 - full convolution : 모든 인풋 픽셀이 k(커널 크기) 번 방문되도록 zero-padding, 아웃풋의 결과는 m+k-1을 가지게 된다. 이러한 경우 아웃풋의 가장자리가 더 적은 input pixel에 의해 영향을 받는다. 이는 모든 position에서 잘 작동하는 하나의 커널을 학습하는 것을 어렵게 만든다. 
 
 ### unshared convolution
 
어떤 경우 convolution 대신 locally fc layer를 사용하고 싶을 수 있습니다. 이러한 경우 adjacency matrix는 같지만 각각의 connection이 각자의 weight를 가지게 됩니다. 이 경우 $\mathbf{W}$ 는 6D tensor가 됩니다. $\mathbf{W}$ 는 i 아웃풋 채널, j 아웃풋 행, k 아웃풋 열, l 인풋 채널, m 인풋의 행, n 인풋의 열이 됩니다. 

이는 종종 <b>unshared convolution</b> 이라고 불리며 location간에 파미터를 공유하지 않는 convolution과 비슷하기 때문입니다. 

locally connected layer는 각 feature가 local한 특징을 가지지만 모든 location에서 같은 feature일 필요가 없을때 유용합니다. 예를 들어 이미지에서의 face를 인식하고 싶다면 우리는 이미지의 bottom 부분에서는 입만을 보고 top 부분에서는 eye feature만을 고려하면 됩니다.

### group convolution

cnn에서의 connection을 더 제한하는 것도 유용합니다. 예를 들어 각 아웃풋 채널 i는 인풋 채널 l의 일부에 대한 함수로 제한할 수 있습니다. 이러한 것을 하는 방법은 아웃풋 채널의 처음 m 채널이 인풋 채널의 처음 n 채널까지만 연결되어 있고 아웃풋 채널의 두번째 m 채널이 인풋 채널의 두번째 m이랑만 연결되어 있도록 하는 것입니다. 이러한 적은 채널끼리만 연결되도록 모델링 하는 것은 적은 파라미터를 사요하게 하여 메모리 사용량과 통계적 효율성을 높입니다. 또한 forward, back-propagation에 사용되는 연산량을 줄여주게 됩니다. 이러한 연산 효율성은 히든 유닛의 갯수를 줄이지 않고도 group convolution을 통해 달성될 수 있습니다.

### Tiled convolution

Tiled convolution은 locally connected layer와 cnn 간의 타협점을 제공합니다. 모든 spatail location에 대해서 다른 파라미터를 학습하는 것이 아니라 spatail space에서 움직일때마다 set of kernel 에서 차례대로 하나씩 꺼내가면서 학습합니다. 이 의미는 근처의 location들은 다른 필터를 가지면서 파라미터에 필요한 메모리는 set of kernel size만큼만 늘어나게 됩니다. 

locally connected layer, tiled convolution은 max-pooling과 함께 사용할때 재밌는 특징을 가집니다. 한 레이어의 각 unit들은 다른 필터에 의해 유도된 것이고 이는 다른 feature를 나타냅니다. 만약 필터가 underlying feature에 대한 다른 transformed version을 하습했다면 max-pooled unit은 어떤 특정한 transformation에 대해 invariant하게 됩니다. 즉 cnn layer가 특정한 translation에 대해 invariant하게 됩니다.

cnn을 구현하기 위해서는 다른 operation들도 필요로 합니다. 학습을 하기 위해서는 gradient wrt ouput이 주어졌을때의 gradient wrt kernel을 구해야만 합니다. 매우 간단한 경우에는 gradient를 구하기 위해 convolution 연산을 사용하면 됩니다. 그러나 stride가 1이상인 경우에는 convolution 연산을 사용하지 못합니다.

convolution은 선형 연산이고 그래서 매트릭스 곱으로 표현 가능합니다 (인풋을 flat 하고 곱해지는 매트릭스는 매우 sparse matrix) 이 매트릭스는 convolution kernel의 함수를 포함하고 있습니다. 매트릭스는 매우 sparse하며 커널의 값들이 매트릭스의 부분에 복사됩니다. 이러한 관점은 cnn을 구현하는데 필요한 다른 연산들을 유도하는데 도움을 줍니다.

위에서 얻어진 매트릭스의 transpose된 것을 곱하는 것 또한 하나의 operation 입니다. 이 operation은 cnn의 back-prop error를 구하는데 필요하며 이 때문에 cnn을 학습하는데 하나이상의 레이어가 필요합니다. (뭔소리야?) 또한 이 operation은 히든 유닛으로 부터 visible unit을 재구성할때도 필요합니다. visible unit을 재구성하는 것은 이 책의 파트3에서 주로 다룹니다. 

transposed convolution은 autoencoer와 같은 모델의 convolutional version을 만드는데 필요합니다. kernel gradient operation 처럼 input gradient operation 도 몇몇 경우는 convolution으로 구할 수 있지만 일반적으론 아니며 새로운 연산이 필요합니다. fowrad propagation을 할때 이러한 transpose operation은 주의가 필요합니다. transpose operation을 통한 아웃풋의 크기는 zero-padding과 convolution stride에 의해 달라집니다. 즉 transpose operation은 원래 인풋 사이즈가 무엇이였는지를 저장해놔야 합니다.

convolution, back-prop from output to weights, back-props from output ot inputs 은 cnn의 모든 gradient를 구하기에 충분하고 어떠한 cnn도 학습할 수 있습니다. Goodfellow는 일반적인 다차원행렬, multi-example의 경우에 대한 수식을 유도하였고 어떻게 이러한 것들이 작동하는지에 대해 설명합니다. 우리는 여기서 2차원, 1 sample의 버젼을 소개합니다..

cnn을 stride convolution kernel $\mathbf{V}$ 과 multi-channel image $\mathbf{V}$ stried s 를 통해 학습한다고 생각해 봅시다. 우리가 원하는 것은 loss J(V,K) 를 줄이는 것입니다. forward propagation 동안 우리는 c(K,V,s) 를 계산해야 하며 이는 네트워크 전체에 걸쳐 propagate되고 cost function J를 계산하는데 사용됩니다. back-prop 동안 tensor G , $\mathbf{G}_{i,j,k} = \frac{\partial }{\partial Z_{i,j,k}} J(\mathbf{V}, \mathbf{K})$ 를 받게 됩니다.

네트워크를 학습시키기 위해서 gradient wrt weight in kernel을 계산해야 하고 다음과 같이 계산할 수 있습니다. 

## $g(\mathbf{G}, \mathbf{V}, s)_{i,j,k,l} = \frac{\partial }{\partial \mathbf{K}_{i,j,k,l}}J(\mathbf{V},\mathbf{K}) =\sum_{m,n}\mathbf{G}_{i,m,n}\mathbf{V}_{j,(m-1)*s+k, (n-1)*s+l}$

이 레이어가 네트워크의 booth layer가 아니라면 gradient wrt V 에 대해 계산해야 합니다. 이는 error는 더 밑으로 back-prop 시켜 줍니다. 

<b>인풋이 알려져 있고 고정된 크기라면 각 location마다 bias를 따로 학습하는 것이 가능합니다. bias를 분리하는 것은 파라미터를 늘리므로 통계적 효율성을 약간 줄이지만 네트워크가 각 location마다의 다른 통계적 특징들을 설명할 수 있게 합니다. </b> 예를들어 same padding을 사용할때 아웃풋의 edge 부분은 더 적은 값을 가지므로 bias가 커지는 경향이 있을 것입니다.

### 인풋, 아웃풋 사이즈가 같은 경우에.. transposed matrix를 이용하여 gradient를 구할 수 있는데.. 다른 경우는 못구한다..!! 그래서 위와 같은 수식들을 통해 gradient를 구한다...??? (솔직히 잘 이해안간다)

## 9.6 Structured Outputs

pixel-wise labeling을 위한 전략중 하나는 image pixel-level label을 먼저 생성한후 이웃한 픽셀간의 상호작용을 통해 initial guess를 refine 하는 것입니다. 이러한 조정 단계를 거치는 것은 웨이트를 공유한 conv layer를 여러번 수행하는 것과 같습니다. 레이어 간에 웨이트가 공유되는 연속적인 conv layer를 수행하는 것은 특별한 형태의 RNN을 수행하는 것과 같습니다. 

<b> Figure 9.17</b> : pixel labeling을 위한 recurrent convolutional network의 예입니다. 인풋은 Image Tensor입니다. 목표는 각 픽셀에 대한 class probability distribution을 예측하는 tensor of label $\hat y$을 예측하는 것입니다. rnn은 반복적으로 추정된 $\hat y$ 을 refine하며 이전의 $\hat y$을 인풋으로 받아 새로운 추정을 하게 됩니다. 같은 파라미터가 매번 타임 스탭마다 사용되며 이러한 estimation은 원하는 만큼 반복가능합니다. 그림에서 보면 U,V 는 $\hat y$을 추정하기 위해 convolve되며 추정된 $\hat y$에 W를 convole되고 인풋 representation과 합쳐져서 hidden state를 만들고 이를 가지고 다음 스탭의 $\hat y$를 추정!! 

### 신기한게 convolution만을 가지고 rnn처럼 만들 수 있다는 것이네.

## 9.8 Efficient Convolution Algorithms

커널사이즈가 큰 경우 FT를 이용한 convolution의 구현이 discreate convolution의 구현보다 빠르다. 

dxd 차원의 커널이 d차원의 하나의 벡터의 outer product를 통해 나타낼 수 있을때 이를 sperable kernel이라고 부릅니다. 커널이 separable하다면 naive discreate convolution은 비효율적입니다. 2개의 축에 대해 하나의 d차원 벡터를 convolution하는 것과 동일합니다. 이러한 sepearable 한 커널을 통한 접근은 원래의 커널보다 연산효율이 좋습니다. 또한 이 커널은 더 적은 파라미터를 가집니다. 하지만 모든 convolution이 sepearble하게 표현될 수 있는 것은 아닙니다.



## 9.9 Random or Unsupervised Features

Random filter는 cnn에서 놀랍도록 잘 작동합니다. Saxe는 convolution, 그후 pooling의 구조가 랜덤한 필터인 경우에도 frequency selective, translation invariant 하다는 것을 보였습니다. (그래서... 우리가 랜덤하게 cnn을 초기화 해도 되는것인가..? 그렇다면..  fc-layer는??)

## 9.10 The Neuroscientific Basis for Convolutional Networks

V1 영역은 뇌에서의 visual input에 대한 처리가 진행되는 시작 단계입니다. 뇌의 입장에서는 이미지는 안구에 닿는 빛으로 형성되어 있으며 retina를 자극하고 빛에 민감한 시각세포를 자극합니다. 레티나에 있는 뉴런들은 이미지에 대한 간단한 전처리를 담당하며 이미지의 representation을 완전히 바꾸지는 않습니다. 이미지는 optic nerve를 지나 lateral geniculate nucleus로 이동하게 됩니다. 여기서 언급된 optic nerve, lateral은 단지 신호를 눈에서 v1영역으로 보내주는 역할을 한다는 것만 알아둡시다.

- V1 영역은 spatial 하게 정리되어져 있습니다. v1영역은 이미지가 들어오면 그에 해당하는 2차원 구조의 영역을 retina에 가지고 있습니다.(receptive field가 있다는 말인듯) 예를 들어 빛이 retina의 중간 아래에 들어오면 v1영역의 중간 정도에만 영향을 끼치게 됩니다. cnn은 2차원 map의 형태로 feature를 가짐으로써 이러한 특징을 가지게 됩니다. (cnn에도 똑같이 receptive field가 있다는 말인듯)

- V1은 많은 simple cell을 가지고 있습니다. simple cell의 활성화는 이미지의 작은 부분에 대한 linear function으로 볼 수 있습니다. cnn의 detector units (활성화 유닛) 은 simple cell의 특징을 따르기 위해 설계되어졌습니다.

-  v1은 많은 complex cell을 가지고 있습니다. 이 세포들은 simple cell이 반응하는 것과 비슷하지만 invariant to small shift 합니다. (특정 방향으로 움직이는 선분에 가장 잘 반응하는 특성을 갖는 시각 피질의 뉴런 즉... 방향에 따라서만 활성화되지 shift랑은 상관이 없다는듯) 이는 cnn에서의 pooling에 동기가 되었습니다. complex cell은 약간의 밝기 변화에 invariant 하며 이는 spatial location pooling으로는 포착되기 힘듭니다. 밝기 변화에 invarinat한 특성을 같기 위해 maxout과 같은 cross-channel pooling의 동기가 되었습니다.

신경과학자들은 medial temporal lobe neuron에 보다 복잡한 형태의 grandmother cell과 같은 것이 있다는 것을 보였습니다. 이는 cnn과 달리 매우 일반화를 잘합니다. 

cnn의 마지막 레이어의 레이어의 feature와 가장 가까운 곳은 inferotemporal cortex라고 불리는 영역입니다. 사람이 객체를 볼때 전기자극은 retina -> LGN -> V1 -> V2 -> V4 -> IT 를 거치게 됩니다. 이러한 처리는 100ms안에 일어납니다. 사람에게 객체를 더 오랜시간동안 쳐다보게 하면 이러한 정보는 top-down process를 통해 backward flow를 형성하여 low level brain area activation을 업데이트 합니다. 그러나 100ms 동안 한번만 쳐다보도록 한다면 그 본 순간에만 활성화 됩니다. IT 영역은 CNN과 매우 비슷하다고 알려져 있습니다. cnn을 통해 IT의 firing rate를 예측할 수 있으며 object recognition task에서 인간과 매우 비슷하게 작동합니다. 

### human vision system VS cnn

- 인간의 눈은 대부분 low resolution이며 fovea만 high resolution이다. 우리가 전체의 장면을 high resolution으로 느끼는 것은 뇌의 subconscious의 영역에 의해 일어난 착시이며 이 착시는 작은 영역을 흘끗흘끗 쳐다본 것들을 합쳐 high resolution처럼 느끼게 한다. 반면에 대부분의 cnn 은 full resolution을 인풋으로 받습니다. 인간은 saccade(안구운동) 을 통해 장면의 중요한 부분에 주의를 집중합니다. 이와 비슷한 것이 NLP, Computer vision에서의 attention 매카니즘 입니다.

- 인간의 vision system은 vision뿐만 아닌 hearing, mood, though와 모두 통합되어져 있습니다. 반면 cnn은 순수하게 vision만을 인풋으로 받습니다.

- 인간의 vision system은 객체를 인식하는것 이상의 일을 할 수 있씁니다. 인간은 많은 객체가 포함된 장면을 이해할 수 있으면 그들간의 관계를 이해할 수 있습니다. 또한 인간의 몸과 세계와 상호작용하기 위한 3-D geometric information을 처리합니다. 반면 cnn은 더 복잡한 문제에 대해 적용되기 시작했지만 아직 초기단계입니다.

- v1과 같은 영역은 더 상위 영역으로 부터 강한 backward feedback을 같습니다. 딥러닝에서의 feed back은 탐구되어지고 있지만 아직 부족합니다.

- IT 영역의 feedforward rate 와 cnn feature는 비슷한 feature를 처리하지만 같은 연산을 통해 비슷한 feature를 처리하는지는 명확하지 않습니다. 뇌는 아마 다른 activation, pooling 함수를 사용할 것입니다. 개별적인 뉴런의 activation은 하나의 linear filter를 통해 잘 표현되지 못합니다. 최근의 연구는 v1이 multiple quadratic filter를 가진다고 주장합니다. 우리가 simple cell, complex cell이라고 구분한것은 잘못된 것일 수 있습니다. simple cell, complex cell은 같은 종류의 cell이며 단지 그들의 파라미터가 달라 simple cell, complex cell이라고 불리는 것일 수 있습니다.

<b>reverse correlation</b> 각 뉴런에 전극을 꼽고 이미지를 줬을때의 전기자극을 측정하고 이미지를 인풋으로하는 linear model을 학습하여 뉴런의 웨이트를 학습하는 방법

### reverse correlation으로 포유동물의 뉴런을 분석한 결과

reverse correlation을 해봤더니 V1의 대부분의 뉴런들의 웨이트들은 일종의 Gabor function 이라는 것을 발견했습니다.

Gabor function은 이미지에서의 2d point에 대한 weight를 기술합니다. 우리는 이미지를 2d coordinate 에 대한 함수 I(x,y)로 생각할 수 있습니다. 이와 마찬가지로 simple cell 또한 x 좌표의 집합, y좌표의 집합에 대한 sampling된 이미지에다가 weight를 곱한 형태로 볼 수 있습니다. 이러한 관점에서 simple cell의 response는 다음과 같습니다. 

#### $s(I) = \sum_{x \in X}\sum_{y \in Y} w(x,y) I(x,y)$

여기서 w(x,y) 가 gabor function의 형태라면 다음과 같습니다.

#### $ w(x,y ; \alpha, \beta_x, \beta_y, f, \phi, x_0, y_o, \tau)=\alpha exp(-\beta_x x^{'2}-\beta_y y^{'2})cos(fx^{'} + \phi)$

#### $x^{'} = (x-x_0)cos(\tau) + (y-y_0)sin(\tau) \\
y^{'} = -(x-x_0)sin(\tau) + (y-y_0)cos(\tau)$

gabor function안의 파라미터는 gabor function의 특징을 결정합니다. 
파라미터 $x_0, y_0, \tau$ 는 coordinate system을 결정합니다. x,y를 x_0, y_0 만큼 trasnlation하고 tau 만큼 rotation한 형태가 x\', y\'가 됩니다. (sin,cos의 형태는 회전행렬을 생각해보면 알 수 있다.) simple cell로 생각해보면 point(x_0, y_0) 을 중심으로 tau 만큼 rotation 한 곳에대한 밝기에 대한 변화에 반응을 할 것입니다. 

garbor function을 x\', y\'에 대한 함수로 바라보면 x\'축을 움직임에 따라서 w의 반응이 변화될 것입니다. w의 식에는 gaussian, cosine function이 있는 것이 중요합니다.

gaussian factor인 $\alpha exp(-\beta_x x^{'2}-\beta_y y^{'2})$은 simple cell의 일종의 gating term으로 볼수 있고 이는 x\', y\'의 근처에서만 반응을 할 것인지 좀더 멀리까지 반응 할 것인지를 결정합니다. 즉 cell의 receptive field를 결정합니다. scaling factor alpha는 simple cell의 총 반응을 조절하며 beta는 얼마나 빨리 receptive feild가 수축할 것인지를 결정합니다. 

cosine factor는 simple cell이 x\'축으로 밝기 변화에 어떻게 반응 할 것인지를 결정합니다. 파라미터 f는 주기를 결정하며 phi는 위상을 결정합니다. (왜 x축에대해서 밖에 위상과 주기 파라미터가 있을까?? 아마 y축에 대해 하고 싶으면.. rotation하면 되기 때문인듯 ?)

<b>simple cell은 specific spatial frequency of brightness, direction, location에 대해 반응합니다.</b> simple cell은 웨이트와 이미지의 밝기가 같은 위상일대 가장 잘 활성화되며 반대 위상일때 덜 활성화 됩니다. 

complex cell은 두개의 simple cell의 반응인 2d vector의 l2 norm으로 볼 수 있습니다. 

#### $c(I(x,y)) = \sqrt{s_o(I(x,y))^2 + s_1(I(x,y))^2}$

s1, s2의 phi를 제외한 모든 파라미터가 같은 경우일떄.. phi는 1/4 quarter이내로 차이가 나야 한다. 이러한 경우 s0, s1은 일종의 <b>quadrature pair</b> 의 형태가 됩니다. 이렇게 정의된 complex cell은 gaussian reweighted image가 높은 주기의 주기신호를 포함한다면 (x_0, y_0) 근처에서 tau 방향으로!!! 하여튼 졸라 높은 주파수가 tau 방향으로 이미지에 있을때.. 반응을 보입니다. 

즉 complex cell은 tau 방향의 small translation에 invariant 해지거나 검은색을 흰색으로 흰색을 검은색으로만 바구게 된다..( 뭔소리야?? 흠... tau방향으로는 고주파일때만 반응하니까... small translation에는 invariant해지고.. 뒤에 검은색 흰색 어쩌구는 뭔소리인지 모르겠다.)

### - v1의 대부분의 뉴런들은 일종의 gabor filter더라
### - simple cell은 gabor function이고 complex cell은 두개의 simple cell의 함수로 볼 수 있더라 